This was the notebook that generated the other 3 folds (after fld 1 had been generated in the main training script on the first run of training). Throught this script, only the images in the train/validation section are touched, leaving the test set images the same and unseen by the model during training and validation.

In [ ]:
path_to_APS360_Project = "/content/drive/MyDrive/"


In [ ]:
import torch
import numpy as np


import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torch.utils import data

from skimage.io import imread
import os

import time
import matplotlib.pyplot as plt

from PIL import Image
import albumentations as A
import random
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
inps = os.listdir(os.path.join(path_to_APS360_Project, "APS360_Project/Dataset/t_image"))
len(inps)

647

In [ ]:
img_root = os.path.join(path_to_APS360_Project, "APS360_Project/Dataset/t_image") #"/content/drive/MyDrive/APS360_Project/Dummy_data/images"
mask_root = os.path.join(path_to_APS360_Project,"APS360_Project/Dataset/t_masked") # "/content/drive/MyDrive/APS360_Project/Dummy_data/masks"
# dset = SegmentationDataSet(inps, img_root, mask_root, train_aug)

In [ ]:
fold1train = []

In [ ]:
# s = "347_1.png"
# s.split("_1")

['347', '.png']

In [ ]:

for f in os.listdir("/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Train/images"):
  if ("_1" in f):
    f = f.split("_1")
    f = f[0] + f[1]
    fold1train.append(f)
  

In [ ]:
len(fold1train)

388

In [ ]:
fold1valid = os.listdir("/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Valid/images")
len(fold1valid)

129

In [ ]:
fold1train.extend(fold1valid)
len(fold1train)

517

In [ ]:
valid_size = len(fold1valid)
valid_size


129

In [ ]:
not_test = np.array(fold1train)
not_test.shape

(517,)

In [ ]:
#generating fold 2:
fold2train = np.concatenate((not_test[:2*valid_size], not_test[3*valid_size:]))
fold2valid = not_test[2*valid_size:3*valid_size]
print(fold2train.shape)
print(fold2valid.shape)


(388,)
(129,)


In [ ]:
foldimg = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Train/images/"
foldmask = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Train/masks/"

for input_ID in fold2train:
  # print(input_ID)
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(foldimg+idx+"_1.png", img_1)
  cv2.imwrite(foldmask+idx+"_1.png", mask_1)

  aug1 = A.HorizontalFlip(p=1.0)
  aug = aug1(image=img_1, mask=mask_1)
  img_2 = aug["image"]
  mask_2 = aug["mask"]
  cv2.imwrite(foldimg+idx+"_2.png", img_2)
  cv2.imwrite(foldmask+idx+"_2.png", mask_2)

  aug2 = A.VerticalFlip(p=1.0)
  aug = aug2(image=img_1, mask=mask_1)
  img_3 = aug["image"]
  mask_3 = aug["mask"]

  cv2.imwrite(foldimg+idx+"_3.png", img_3)
  cv2.imwrite(foldmask+idx+"_3.png", mask_3)

  img_4 = cv2.rotate(img_1, cv2.ROTATE_90_CLOCKWISE)
  mask_4 = cv2.rotate(mask_1, cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(foldimg+idx+"_4.png", img_4)
  cv2.imwrite(foldmask+idx+"_4.png", mask_4)


  img_5 = cv2.rotate(img_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  mask_5 = cv2.rotate(mask_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  cv2.imwrite(foldimg+idx+"_5.png", img_5)
  cv2.imwrite(foldmask+idx+"_5.png", mask_5)

  img_6 = cv2.rotate(img_1, cv2.ROTATE_180)
  mask_6 = cv2.rotate(mask_1, cv2.ROTATE_180)
  cv2.imwrite(foldimg+idx+"_6.png", img_6)
  cv2.imwrite(foldmask+idx+"_6.png", mask_6)


In [ ]:
validimgs = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Valid/images/"
validmasks = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Valid/masks/"
for input_ID in fold2valid:
  # input_ID = train_inps[0]
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(validimgs+input_ID, img_1)
  cv2.imwrite(validmasks+input_ID, mask_1)

In [ ]:
len(os.listdir(validmasks))

129

In [ ]:
#generating fold 3
fold3train = np.concatenate((not_test[:1*valid_size], not_test[2*valid_size:]))
fold3valid = not_test[1*valid_size:2*valid_size]
print(fold3train.shape)
print(fold3valid.shape)


(388,)
(129,)


In [ ]:
foldimg ="/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold3/Train/images/" #"/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Train/images/"
foldmask = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold3/Train/masks/"

for input_ID in fold3train:
  # print(input_ID)
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(foldimg+idx+"_1.png", img_1)
  cv2.imwrite(foldmask+idx+"_1.png", mask_1)

  aug1 = A.HorizontalFlip(p=1.0)
  aug = aug1(image=img_1, mask=mask_1)
  img_2 = aug["image"]
  mask_2 = aug["mask"]
  cv2.imwrite(foldimg+idx+"_2.png", img_2)
  cv2.imwrite(foldmask+idx+"_2.png", mask_2)

  aug2 = A.VerticalFlip(p=1.0)
  aug = aug2(image=img_1, mask=mask_1)
  img_3 = aug["image"]
  mask_3 = aug["mask"]

  cv2.imwrite(foldimg+idx+"_3.png", img_3)
  cv2.imwrite(foldmask+idx+"_3.png", mask_3)

  img_4 = cv2.rotate(img_1, cv2.ROTATE_90_CLOCKWISE)
  mask_4 = cv2.rotate(mask_1, cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(foldimg+idx+"_4.png", img_4)
  cv2.imwrite(foldmask+idx+"_4.png", mask_4)


  img_5 = cv2.rotate(img_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  mask_5 = cv2.rotate(mask_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  cv2.imwrite(foldimg+idx+"_5.png", img_5)
  cv2.imwrite(foldmask+idx+"_5.png", mask_5)

  img_6 = cv2.rotate(img_1, cv2.ROTATE_180)
  mask_6 = cv2.rotate(mask_1, cv2.ROTATE_180)
  cv2.imwrite(foldimg+idx+"_6.png", img_6)
  cv2.imwrite(foldmask+idx+"_6.png", mask_6)


In [ ]:
len(os.listdir(foldimg))

2295

In [ ]:
validimgs = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold3/Valid/images/"
validmasks = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold3/Valid/masks/"
for input_ID in fold3valid:
  # input_ID = train_inps[0]
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(validimgs+input_ID, img_1)
  cv2.imwrite(validmasks+input_ID, mask_1)

In [ ]:
len(os.listdir("/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold3/Valid/masks"))

2

In [ ]:
#generating fold 4
fold4train = not_test[valid_size:]
fold4valid = not_test[:valid_size]
print(fold4train.shape)
print(fold4valid.shape)


(388,)
(129,)


In [ ]:
foldimg ="/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold4/Train/images/" #"/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold2/Train/images/"
foldmask = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold4/Train/masks/"

for input_ID in fold4train:
  # print(input_ID)
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(foldimg+idx+"_1.png", img_1)
  cv2.imwrite(foldmask+idx+"_1.png", mask_1)

  aug1 = A.HorizontalFlip(p=1.0)
  aug = aug1(image=img_1, mask=mask_1)
  img_2 = aug["image"]
  mask_2 = aug["mask"]
  cv2.imwrite(foldimg+idx+"_2.png", img_2)
  cv2.imwrite(foldmask+idx+"_2.png", mask_2)

  aug2 = A.VerticalFlip(p=1.0)
  aug = aug2(image=img_1, mask=mask_1)
  img_3 = aug["image"]
  mask_3 = aug["mask"]

  cv2.imwrite(foldimg+idx+"_3.png", img_3)
  cv2.imwrite(foldmask+idx+"_3.png", mask_3)

  img_4 = cv2.rotate(img_1, cv2.ROTATE_90_CLOCKWISE)
  mask_4 = cv2.rotate(mask_1, cv2.ROTATE_90_CLOCKWISE)
  cv2.imwrite(foldimg+idx+"_4.png", img_4)
  cv2.imwrite(foldmask+idx+"_4.png", mask_4)


  img_5 = cv2.rotate(img_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  mask_5 = cv2.rotate(mask_1, cv2.ROTATE_90_COUNTERCLOCKWISE)
  cv2.imwrite(foldimg+idx+"_5.png", img_5)
  cv2.imwrite(foldmask+idx+"_5.png", mask_5)

  img_6 = cv2.rotate(img_1, cv2.ROTATE_180)
  mask_6 = cv2.rotate(mask_1, cv2.ROTATE_180)
  cv2.imwrite(foldimg+idx+"_6.png", img_6)
  cv2.imwrite(foldmask+idx+"_6.png", mask_6)


In [ ]:
validimgs = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold4/Valid/images/"
validmasks = "/content/drive/MyDrive/APS360_Project/Dataset_Expanded/Fold4/Valid/masks/"
for input_ID in fold4valid:
  # input_ID = train_inps[0]
  idx = input_ID.split('.')[0] 
  img_path = os.path.join(img_root, input_ID) #may need to add in .png at end
  img_1 = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)

  mask_path = os.path.join(mask_root, input_ID) 
  mask_1 = cv2.imread(mask_path,  cv2.IMREAD_GRAYSCALE)#Image.open(mask_path).convert("L") #grayscale

  cv2.imwrite(validimgs+input_ID, img_1)
  cv2.imwrite(validmasks+input_ID, mask_1)